In [7]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_68d6b15382848190abfbf38c4df9d58405059e90f8d17748', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_sRBaJv9F1X4Mo5Jyy1b37eMp', name='get_horoscope', type='function_call', id='fc_68d6b157e01c8190a689c1156de3351905059e90f8d17748', status='completed'), {'type': 'function_call_output', 'call_id': 'call_sRBaJv9F1X4Mo5Jyy1b37eMp', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_68d6b158781c8190a2bb3f7679a9ae9f05059e90f8d17748",
  "created_at": 1758900568.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_68d6b1595b7